<a href="https://colab.research.google.com/github/SampMark/ETL-de-dados-da-PNP/blob/main/ETL_of_PNP_Data___Pipeline_for_BigQuery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Extração da Plataforma Nilo Peçanha (PNP), filtragem, tratamento e armazenamento em Big Query dos dados de Instituições Federais de Ensino (IFES)**

## **Fluxo de Execução**
1. **Etapa 1: Instalação e Autenticação**: Instalação de bibliotecas e conecção ao Google Drive.
2. **Etapa 2: Configuração do Pipeline**: definição dos parâmetros principais, como os tipos de dado da tabela, o período da extração (anos) e a(as) instituição(ões) a ser(em) filtrada(s).
3. **Etapa 3: Download dos dados compactados da PNP**: nessa etapa é efetuado o download os arquivos compactados (**`.gz`**) da PNP para o Google Drive com base nas configurações da etapa anterior.
4. **Etapa 4: Análise dos cabeçalhos dos CSVs ao longo dos anos**: nesta etapa crucial o script descompacta os arquivos, lê o cabeçalho de cada um e os exibe em uma tabela comparativa. Por fim, sugere uma lista de colunas que são comuns a todos os arquivos **`.csv`**, para decisão sobre quais variáveis manter.
5. **Etapa 5: Processamento da extração do Big Data**: após a definição da lista final de colunas a serem mantida é executado o processamento, que resulta na unificação e filtragem dos dados num _DataFrame_ Pandas.
6. **Etapa 6: Limpeza e tratamento de dados Específicos**: em seguida, são aplicadas um conjunto de funções para limpeza e padronizações dos dados.
7. **Etapa 7: Exportação para BigQuery**: por fim, o _DataFrame_ tratado é salvo numa tabela do banco de dados do **Google BigQuery**.
<img src="https://www2.ifal.edu.br/noticias/ifal-se-destaca-na-eficiencia-academica-dos-institutos-federais-do-nordeste/plataforma-nilo-pecanha/@@images/98c1a2a4-6c59-436f-bdce-effa7ae4d539.jpeg" alt="Logo da Plataforma Nilo Peçanha" width="250"/>

In [1]:
# -*- coding: utf-8 -*-
"""
Extração de microdados da PNP para Big Query

Este notebook automatiza o fluxo de trabalho com os microdados extraídos da Plataforma Nilo Peçanha (PNP),
permitindo a extração de diferentes tabelas e a análise de seus cabeçalhos em cada anos.
"""

# @title **ETAPA 1: Instalação de dependências, importações e autenticação do usuário no Google Drive**

# Instalação de Dependências
!pip install gspread gspread-dataframe oauth2client pandas-gbq --quiet
print("Dependências instaladas com sucesso!")

Dependências instaladas com sucesso!


In [2]:
# Importação de bibliotecas
import re
import pandas as pd
import requests
import gspread
import gzip
import shutil
import ipywidgets as widgets
from IPython.display import display, HTML
from google.colab import auth, drive
from google.auth import default
from pathlib import Path
from typing import List, Dict, Set

In [3]:
# Autenticação e Montagem do Google Drive
try:
    auth.authenticate_user()
    creds, _ = default()
    gc = gspread.authorize(creds)
    drive.mount('/content/drive')
    print("\nAutenticação e montagem do Google Drive realizadas com sucesso!")
except Exception as e:
    print(f"Ocorreu um erro durante a autenticação ou montagem do Drive: {e}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

Autenticação e montagem do Google Drive realizadas com sucesso!


In [4]:
# @title **ETAPA 2: Definição das Funções Principais**

def download_pnp_data(table_name: str, start_year: int, end_year: int, force_update: bool):
    """
    Baixa os arquivos da PNP para uma pasta específica no Google Drive.
    O nome do arquivo e a pasta de destino são baseados no table_name.
    """
    drive_folder = Path(f'/content/drive/MyDrive/Coisas do IFRN/Prodes/Indicadores/PNP/{table_name.capitalize()}')
    drive_folder.mkdir(parents=True, exist_ok=True)
    print(f"Verificando arquivos na pasta do Google Drive: {drive_folder}")

    base_url = "https://d236w85zd3t8iw.cloudfront.net/pnp-tests/microdados"

    for year in range(start_year, end_year + 1):
        file_name = f"microdados_{table_name}_{year}.csv.gz"
        url = f"{base_url}/{year}/{file_name}"
        destination = drive_folder / file_name

        if not force_update and destination.exists():
            print(f"✔ O arquivo para {year} ('{file_name}') já existe. Usando o cache do Drive.")
            continue

        try:
            print(f"⬇ Baixando dados para {year} de {url}...")
            with requests.get(url, stream=True) as r:
                r.raise_for_status()
                with open(destination, 'wb') as f:
                    shutil.copyfileobj(r.raw, f)
            print(f"✔ Download de {year} concluído com sucesso.")
        except requests.exceptions.RequestException as e:
            print(f"❌ Falha ao baixar o arquivo para {year}. Erro: {e}. O arquivo pode não existir para este ano.")

def decompress_gz_to_csv(gz_path: Path, out_dir: Path) -> Path:
    """Descompacta cada arquivo .gz para a pasta de trabalho"""
    out_dir.mkdir(parents=True, exist_ok=True)
    # Remove a extensão .gz para obter o nome do arquivo CSV
    csv_out_path = out_dir / gz_path.with_suffix("").name
    print(f"Descompactando: {gz_path.name} -> {csv_out_path.name}")
    with gzip.open(gz_path, "rb") as f_in, open(csv_out_path, "wb") as f_out:
        shutil.copyfileobj(f_in, f_out)
    return csv_out_path

def get_header(path: Path, sep: str = ';') -> List[str]:
    """Lê o cabeçalho de cada arquivo CSV descompactado"""
    try:
        return list(pd.read_csv(path, nrows=0, sep=sep, engine='python').columns)
    except Exception as e:
        print(f"Erro ao ler o cabeçalho de {path.name}: {e}")
        return []

def analyze_and_compare_headers(csv_paths: List[Path]) -> pd.DataFrame:
    """
    Cria um DataFrame comparativo de cabeçalhos e sugere colunas comuns.
    """
    if not csv_paths:
        print("Nenhum arquivo CSV para analisar.")
        return pd.DataFrame(), []

    headers_dict = {path.name: get_header(path) for path in csv_paths}

    # Identificar colunas comuns
    sets_of_headers = [set(h) for h in headers_dict.values() if h]
    if not sets_of_headers:
        print("Não foi possível ler nenhum cabeçalho.")
        return pd.DataFrame(), []

    common_columns = sorted(list(sets_of_headers[0].intersection(*sets_of_headers[1:])))

    # Criar DataFrame para comparação visual
    all_columns = sorted(list(set.union(*sets_of_headers)))
    comparison_data = {}
    for col in all_columns:
        comparison_data[col] = [("✔" if col in headers_dict.get(fname, []) else "❌") for fname in headers_dict.keys()]

    comparison_df = pd.DataFrame(comparison_data, index=headers_dict.keys()).transpose()

    print("\n--- Análise de Cabeçalhos Concluída ---")
    print("A tabela abaixo mostra quais colunas estão presentes (✔) ou ausentes (❌) em cada arquivo.")
    display(HTML(comparison_df.to_html()))

    print("\n--- Sugestão de Colunas Comuns ---")
    print(f"Foram encontradas {len(common_columns)} colunas presentes em TODOS os arquivos do período:")
    # Imprime a lista formatada para ser copiada e colada
    print("\nfinal_columns_to_keep = [")
    for col in common_columns:
        print(f"    '{col}',")
    print("]")

    return comparison_df, common_columns

def process_to_dataframe(csv_paths: List[Path], columns_to_keep: List[str], institutions: List[str], chunksize: int = 100000, sep: str = ';'):
    """
    Unifica, filtra e concatena os CSVs em um único DataFrame,
    mantendo apenas a lista de colunas fornecida.
    """
    if not csv_paths:
        raise RuntimeError("Nenhum arquivo CSV para processar.")
    if not columns_to_keep:
        raise ValueError("A lista 'columns_to_keep' não pode estar vazia.")

    print(f"\nProcessamento iniciado. Serão importadas {len(columns_to_keep)} colunas pré-definidas.")

    # Encontrar coluna da instituição (considerando inconsistências de codificação)
    col_inst = None
    if 'Instituição' in columns_to_keep:
        col_inst = 'Instituição'
    elif 'InstituiÃ§Ã£o' in columns_to_keep:
        col_inst = 'InstituiÃ§Ã£o'

    if institutions and col_inst:
        print(f"Filtrando pela coluna '{col_inst}' com os valores: {institutions}")
    elif institutions:
        print("AVISO: Filtro de instituição solicitado, mas a coluna 'Instituição' não está na lista de colunas a serem mantidas.")

    institution_map = {'Instituto Federal do Rio Grande do Norte': 'IFRN'}
    df_list = []

    for csv_path in csv_paths:
        print(f"Processando e filtrando: {csv_path.name}")
        try:
            # Lê o cabeçalho do arquivo para saber quais colunas ele realmente tem
            actual_header = get_header(csv_path, sep)
            # Usa apenas as colunas da nossa lista que existem neste arquivo
            cols_to_read = [col for col in columns_to_keep if col in actual_header]

            for chunk in pd.read_csv(csv_path, usecols=cols_to_read, chunksize=chunksize, sep=sep, engine='python', on_bad_lines='warn'):
                if col_inst and col_inst in chunk.columns:
                    chunk[col_inst] = chunk[col_inst].replace(institution_map)
                    if institutions:
                        chunk = chunk[chunk[col_inst].isin(institutions)]

                if not chunk.empty:
                    df_list.append(chunk)
        except Exception as e:
            print(f"  ERRO ao processar o arquivo {csv_path.name}: {e}. Pulando este arquivo.")
            continue

    if not df_list:
        print("AVISO: Nenhum dado encontrado para as instituições selecionadas ou os arquivos estavam vazios.")
        return pd.DataFrame(columns=columns_to_keep)

    final_df = pd.concat(df_list, ignore_index=True)
    # Garante que o DataFrame final tenha todas as colunas da lista, preenchendo com NaN as que não existiam
    final_df = final_df.reindex(columns=columns_to_keep)

    print(f"\nProcesso concluído. DataFrame final criado com {len(final_df):,} linhas e {len(final_df.columns)} colunas.")
    return final_df

In [5]:
# @title **ETAPA 3: Configuração do Processo e Download (Opcional)**

# --- Interface Interativa de Configuração ---
style = {'description_width': 'initial'}

# Seleção da tabela
table_name_dropdown = widgets.Dropdown(
    options=['matriculas', 'eficiencia_academica', 'financeiro', 'servidores'],
    value='matriculas',
    description='Tabela de Dados:',
    style=style
)

# Período de anos
start_year_slider = widgets.IntSlider(value=2017, min=2017, max=2024, step=1, description='Ano Inicial:', style=style)
end_year_slider = widgets.IntSlider(value=2024, min=2017, max=2024, step=1, description='Ano Final:', style=style)

# Opção de forçar atualização
force_update_checkbox = widgets.Checkbox(value=False, description='Forçar atualização (baixar novamente os arquivos existentes)', style=style)

# --- Filtro de instituições por código ---
# O campo pede o código numérico da instituição.
institution_code_text = widgets.Text(
    value='26435', # O valor padrão está preenchido com o código do IFRN = '26435'
    description='Códigos das Instituições (Co Inst):',
    style=style,
    layout=widgets.Layout(width='50%')
)

print("--- Configure os Parâmetros do Pipeline ---")
display(table_name_dropdown)
display(widgets.HBox([start_year_slider, end_year_slider]))
display(force_update_checkbox)
display(institution_code_text)

--- Configure os Parâmetros do Pipeline ---


Dropdown(description='Tabela de Dados:', options=('matriculas', 'eficiencia_academica', 'financeiro', 'servido…

Checkbox(value=False, description='Forçar atualização (baixar novamente os arquivos existentes)', style=Descri…

Text(value='26435', description='Códigos das Instituições (Co Inst):', layout=Layout(width='50%'), style=Descr…

In [6]:
# @title **ETAPA 3: Configuração do Processo e Download**

# --- Interface Interativa de Configuração ---
style = {'description_width': 'initial'}

# Seleção da tabela
table_name_dropdown = widgets.Dropdown(
    options=['matriculas', 'eficiencia_academica', 'financeiro', 'servidores'],
    value='matriculas',
    description='Tabela de Dados:',
    style=style
)

# Período de anos
start_year_slider = widgets.IntSlider(value=2017, min=2017, max=2024, step=1, description='Ano Inicial:', style=style)
end_year_slider = widgets.IntSlider(value=2024, min=2017, max=2024, step=1, description='Ano Final:', style=style)

# Opção de forçar atualização
force_update_checkbox = widgets.Checkbox(value=False, description='Forçar atualização (baixar novamente os arquivos existentes)', style=style)

# --- Filtro de instituições por nome ---
# O campo agora aceita um ou mais nomes de instituições, separados por vírgula.
# O valor padrão já inclui as duas variações para o IFRN.
institution_name_text = widgets.Text(
    value='IFRN, Instituto Federal do Rio Grande do Norte',
    description='Nome da Instituição (use vírgula para múltiplos nomes):',
    style=style,
    layout=widgets.Layout(width='70%') # Largura aumentada para melhor visualização
)

print("--- Configure os Parâmetros do Pipeline ---")
display(table_name_dropdown)
display(widgets.HBox([start_year_slider, end_year_slider]))
display(force_update_checkbox)
display(institution_name_text)

--- Configure os Parâmetros do Pipeline ---


Dropdown(description='Tabela de Dados:', options=('matriculas', 'eficiencia_academica', 'financeiro', 'servido…

Checkbox(value=False, description='Forçar atualização (baixar novamente os arquivos existentes)', style=Descri…

Text(value='IFRN, Instituto Federal do Rio Grande do Norte', description='Nome da Instituição (use vírgula par…

In [7]:
# @title **ETAPA 4: Download dos arquivos `.gz` e análise dos cabeçalhos dos `.csv` extraídos**

# 1. Pega os valores dos widgets da Etapa 3
table_name = table_name_dropdown.value
start_year = start_year_slider.value
end_year = end_year_slider.value
force_update = force_update_checkbox.value
# Use the correct widget name and parse the comma-separated string
institutions_str = institution_name_text.value
institutions_list = [inst.strip() for inst in institutions_str.split(',') if inst.strip()]

# 2. Executa o download
download_pnp_data(table_name, start_year, end_year, force_update)

# 3. Prepara os arquivos para a análise
drive_folder = Path(f'/content/drive/MyDrive/Coisas do IFRN/Prodes/Indicadores/PNP/{table_name.capitalize()}')
work_dir = Path.cwd() / "extracted_csvs"
if work_dir.exists(): shutil.rmtree(work_dir)
work_dir.mkdir()

input_files_gz = [drive_folder / f"microdados_{table_name}_{year}.csv.gz" for year in range(start_year, end_year + 1)]
input_files_gz_existing = [f for f in input_files_gz if f.exists()]

all_csvs = []
if input_files_gz_existing:
    for gz_file in input_files_gz_existing:
        all_csvs.append(decompress_gz_to_csv(gz_file, work_dir))
else:
    print("Nenhum arquivo .gz encontrado no Drive para o período e tabela selecionados.")

# 4. Analisa e compara os cabeçalhos
if all_csvs:
    comparison_df, common_columns = analyze_and_compare_headers(all_csvs)
else:
    print("Nenhuma análise de cabeçalho pôde ser feita, pois nenhum arquivo CSV foi descompactado.")

print("\n\n>>> AÇÃO NECESSÁRIA <<<")
print("Copie a lista de colunas comuns sugerida acima (ou edite-a conforme sua necessidade) e cole na célula da 'ETAPA 5' antes de executá-la.")

Verificando arquivos na pasta do Google Drive: /content/drive/MyDrive/Coisas do IFRN/Prodes/Indicadores/PNP/Matriculas
✔ O arquivo para 2017 ('microdados_matriculas_2017.csv.gz') já existe. Usando o cache do Drive.
✔ O arquivo para 2018 ('microdados_matriculas_2018.csv.gz') já existe. Usando o cache do Drive.
✔ O arquivo para 2019 ('microdados_matriculas_2019.csv.gz') já existe. Usando o cache do Drive.
✔ O arquivo para 2020 ('microdados_matriculas_2020.csv.gz') já existe. Usando o cache do Drive.
✔ O arquivo para 2021 ('microdados_matriculas_2021.csv.gz') já existe. Usando o cache do Drive.
✔ O arquivo para 2022 ('microdados_matriculas_2022.csv.gz') já existe. Usando o cache do Drive.
✔ O arquivo para 2023 ('microdados_matriculas_2023.csv.gz') já existe. Usando o cache do Drive.
✔ O arquivo para 2024 ('microdados_matriculas_2024.csv.gz') já existe. Usando o cache do Drive.
Descompactando: microdados_matriculas_2017.csv.gz -> microdados_matriculas_2017.csv
Descompactando: microdados_ma

,microdados_matriculas_2017.csv,microdados_matriculas_2018.csv,microdados_matriculas_2019.csv,microdados_matriculas_2020.csv,microdados_matriculas_2021.csv,microdados_matriculas_2022.csv,microdados_matriculas_2023.csv,microdados_matriculas_2024.csv
Ano,✔,✔,✔,✔,✔,✔,✔,✔
Carga Horaria,✔,✔,✔,✔,✔,✔,✔,✔
Carga Horaria Mínima,✔,✔,✔,✔,✔,✔,✔,✔
Categoria da Situação,✔,✔,✔,✔,✔,✔,✔,✔
Co Inst,✔,✔,✔,✔,✔,✔,✔,✔
Cod Unidade,✔,✔,✔,✔,✔,✔,✔,✔
Cor / Raça,✔,✔,✔,✔,✔,✔,✔,✔
Código da Matricula,✔,✔,✔,✔,✔,✔,✔,✔
Código da Unidade de Ensino - SISTEC,✔,✔,✔,✔,✔,✔,✔,✔
Código do Ciclo Matricula,✔,✔,✔,✔,✔,✔,✔,✔



--- Sugestão de Colunas Comuns ---
Foram encontradas 54 colunas presentes em TODOS os arquivos do período:

final_columns_to_keep = [
    'Ano',
    'Carga Horaria',
    'Carga Horaria Mínima',
    'Categoria da Situação',
    'Co Inst',
    'Cod Unidade',
    'Cor / Raça',
    'Código da Matricula',
    'Código da Unidade de Ensino - SISTEC',
    'Código do Ciclo Matricula',
    'Código do Município com DV',
    'Data de Fim Previsto do Ciclo',
    'Data de Inicio do Ciclo',
    'Data de Ocorrencia da Matricula',
    'Eixo Tecnológico',
    'Faixa Etária',
    'Fator Esforço Curso',
    'Fonte de Financiamento',
    'Idade',
    'Instituição',
    'Matrícula Atendida',
    'Modalidade de Ensino',
    'Município',
    'Mês De Ocorrência da Situação',
    'Nome de Curso',
    'Região',
    'Renda Familiar',
    'Sexo',
    'Situação de Matrícula',
    'Subeixo Tecnológico',
    'Tipo de Curso',
    'Tipo de Oferta',
    'Total de Inscritos',
    'Turno',
    'UF',
    'Unidade de Ensin

In [8]:
# @title **ETAPA 5: Definição das colunas necessárias para extração do `DataFrame`**

# >>> LISTA DE COLUNAS PARA EXTRAÇÃO (COLE AQUI) <<<
# Exemplo baseado na sugestão da etapa anterior.
# Edite esta lista conforme a necessidade.
# Lista de colunas a serem mantidas no DataFrame final, organizadas por categoria e finalidade.

final_columns_to_keep = [

    # --- IDENTIFICADORES E CÓDIGOS ---
    # Códigos únicos utilizados para referenciar instituições, unidades, matrículas e ciclos.
    'Co Inst', 'Instituição', 'Cod Unidade', 'Unidade de Ensino', 'Código da Unidade de Ensino - SISTEC',
    'Código do Ciclo Matricula', 'Código da Matricula',

    # --- DADOS DE LOCALIZAÇÃO ---
    # Variáveis de localização geográfica.
    'Município', 'Código do Município com DV', 'UF', 'Região',

    # --- DADOS DO CURSO ---
    # Variáveis que descrevem as características do curso ofertado.
    'Modalidade de Ensino', 'Tipo de Curso', 'Tipo de Oferta',  'Eixo Tecnológico', 'Subeixo Tecnológico',
    'Nome de Curso', 'Turno', 'Carga Horaria', 'Carga Horaria Mínima', 'Fonte de Financiamento',
    'Fator Esforço Curso', # Fator de ponderação para cálculo de Matrícula Equivalente

    # --- DADOS DO ALUNO (DEMOGRÁFICOS E SOCIOECONÔMICOS) ---
    # Variáveis com informações pessoais e socioeconômicas do estudante.
    'Sexo', 'Cor / Raça', 'Idade', 'Faixa Etária', 'Renda Familiar',

    # --- DADOS DA MATRÍCULA ---
    # Variáveis que detalham o status e o período do vínculo do aluno com o curso.
    'Ano', 'Matrícula Atendida', # Variável booleana indicando se a matrícula é válida para o ano
    'Situação de Matrícula', 'Categoria da Situação', 'Data de Inicio do Ciclo', 'Data de Fim Previsto do Ciclo',
    'Data de Ocorrencia da Matricula', 'Mês De Ocorrência da Situação',

    # --- DADOS DE VAGAS E INGRESSO ---
    # Variáveis relacionadas ao processo seletivo, número de vagas e forma de entrada.
    'Total de Inscritos', 'Forma de ingresso', 'Vagas Ofertadas', 'Vagas Regulares AC',
    'Vagas Regulares l1', 'Vagas Regulares l2', 'Vagas Regulares l5', 'Vagas Regulares l6',
    'Vagas Regulares l9', 'Vagas Regulares l10', 'Vagas Regulares l13', 'Vagas Regulares l14',
    'Vagas Extraordinárias AC', 'Vagas Extraordinárias l1', 'Vagas Extraordinárias l2',
    'Vagas Extraordinárias l5', 'Vagas Extraordinárias l6', 'Vagas Extraordinárias l9',
    'Vagas Extraordinárias l10', 'Vagas Extraordinárias l13', 'Vagas Extraordinárias l14'
]

# --- Execução do Processamento ---
df_final = None # Inicializa a variável
if not final_columns_to_keep:
    print("❌ ERRO: A lista 'final_columns_to_keep' está vazia. Preencha-a com as colunas desejadas e execute novamente.")
elif not all_csvs:
    print("❌ ERRO: Nenhum arquivo CSV foi encontrado para processar. Execute a Etapa 4 primeiro.")
else:
    # Cria o DataFrame final com base na sua seleção de colunas
    df_final = process_to_dataframe(
        csv_paths=all_csvs,
        columns_to_keep=final_columns_to_keep,
        institutions=institutions_list
    )
    display(df_final.head())

# --- Análise do DataFrame Gerado ---
if df_final is not None:
    print("\n--- Análise Detalhada do DataFrame Final ---")
    if df_final.empty:
        print("O DataFrame foi criado, mas está vazio (não contém linhas).")
    else:
        num_rows, num_cols = df_final.shape
        print(f"Dimensões: {num_rows:,} linhas e {num_cols} colunas.")
        print("\nEstrutura e Tipos de Dados:")
        df_final.info()
else:
    print("\nO DataFrame 'df_final' não foi criado. Verifique os erros nas etapas anteriores.")


Processamento iniciado. Serão importadas 56 colunas pré-definidas.
Filtrando pela coluna 'Instituição' com os valores: ['IFRN', 'Instituto Federal do Rio Grande do Norte']
Processando e filtrando: microdados_matriculas_2017.csv
Processando e filtrando: microdados_matriculas_2018.csv
Processando e filtrando: microdados_matriculas_2019.csv
Processando e filtrando: microdados_matriculas_2020.csv
Processando e filtrando: microdados_matriculas_2021.csv
Processando e filtrando: microdados_matriculas_2022.csv
Processando e filtrando: microdados_matriculas_2023.csv
Processando e filtrando: microdados_matriculas_2024.csv

Processo concluído. DataFrame final criado com 369,095 linhas e 56 colunas.


,Co Inst,Instituição,Cod Unidade,Unidade de Ensino,Código da Unidade de Ensino - SISTEC,Código do Ciclo Matricula,Código da Matricula,Município,Código do Município com DV,UF,...,Vagas Regulares l14,Vagas Extraordinárias AC,Vagas Extraordinárias l1,Vagas Extraordinárias l2,Vagas Extraordinárias l5,Vagas Extraordinárias l6,Vagas Extraordinárias l9,Vagas Extraordinárias l10,Vagas Extraordinárias l13,Vagas Extraordinárias l14
0,26435,IFRN,346.0,Campus Nova Cruz,NaN,2449328,83355157,Nordeste,NaN,RN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,26435,IFRN,346.0,Campus Nova Cruz,NaN,2449328,83354097,Nordeste,NaN,RN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,26435,IFRN,346.0,Campus Nova Cruz,NaN,2449328,83354093,Nordeste,NaN,RN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,26435,IFRN,338.0,Campus Natal Central,NaN,1781299,63886076,Nordeste,NaN,RN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,26435,IFRN,346.0,Campus Nova Cruz,NaN,2449328,83354565,Nordeste,NaN,RN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN



--- Análise Detalhada do DataFrame Final ---
Dimensões: 369,095 linhas e 56 colunas.

Estrutura e Tipos de Dados:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 369095 entries, 0 to 369094
Data columns (total 56 columns):
 #   Column                                Non-Null Count   Dtype  
---  ------                                --------------   -----  
 0   Co Inst                               369095 non-null  int64  
 1   Instituição                           369095 non-null  object 
 2   Cod Unidade                           369095 non-null  float64
 3   Unidade de Ensino                     369095 non-null  object 
 4   Código da Unidade de Ensino - SISTEC  307975 non-null  float64
 5   Código do Ciclo Matricula             369095 non-null  int64  
 6   Código da Matricula                   369095 non-null  int64  
 7   Município                             369095 non-null  object 
 8   Código do Município com DV            308474 non-null  float64
 9   UF                   

In [9]:
# @title **Listagem e análise de valores únicos para as colunas indicadas**
if df_final is not None:
    columns_to_list = ['Ano', 'Unidade de Ensino', 'Eixo Tecnológico', 'Subeixo Tecnológico', 'Matrícula Atendida',
                       'Carga Horaria', 'Carga Horaria Mínima', 'Forma de ingresso', 'Faixa Etária', 'Fonte de Financiamento',
                       'Tipo de Oferta', 'Tipo de Curso', 'Situação de Matrícula', 'Categoria da Situação',
                       ]

    for col in columns_to_list:
        if col in df_final.columns:
            # Convert the column to string type before sorting unique values
            unique_values = sorted(df_final[col].astype(str).unique())
            print(f"Valores únicos em ordem alfabética na coluna '{col}':")
            for value in unique_values:
                print(f"- {value}")
            print("-" * 20) # Separador
        else:
            print(f"A coluna '{col}' não foi encontrada no DataFrame 'df_final'.")
else:
    print("DataFrame 'df_final' não encontrado. Execute as etapas anteriores.")

Valores únicos em ordem alfabética na coluna 'Ano':
- 2017
- 2018
- 2019
- 2020
- 2021
- 2022
- 2023
- 2024
--------------------
Valores únicos em ordem alfabética na coluna 'Unidade de Ensino':
- Campus Apodi
- Campus Avançado Jucurutu
- Campus Avançado Lajes
- Campus Avançado Natal Zona Leste
- Campus Avançado Parelhas
- Campus Caicó
- Campus Canguaretama
- Campus Ceará-Mirim
- Campus Ceará-mirim
- Campus Currais Novos
- Campus Ipanguaçu
- Campus João Câmara
- Campus Lajes
- Campus Macau
- Campus Mossoró
- Campus Natal Central
- Campus Natal Cidade Alta
- Campus Natal Zona Norte
- Campus Nova Cruz
- Campus Parelhas
- Campus Parnamirim
- Campus Pau Dos Ferros
- Campus Pau dos Ferros
- Campus Santa Cruz
- Campus São Gonçalo do Amarante
- Campus São Paulo do Potengi
--------------------
Valores únicos em ordem alfabética na coluna 'Eixo Tecnológico':
- Ambiente e Saúde
- Controle e Processos Industriais
- Desenvolvimento Educacional e Social
- Gestão e Negócios
- Informação e Comunicaçã

In [10]:
# @title **Exibir valores únicos da coluna 'Nome de Curso' em df_final**
if 'df_final' in locals() and df_final is not None and 'Nome de Curso' in df_final.columns:
    unique_nomes_curso = sorted(df_final['Nome de Curso'].unique())
    print("Valores únicos na coluna 'Nome de Curso' (df_final):")
    print(f"Total de valores únicos: {len(unique_nomes_curso)}")
    for nome in unique_nomes_curso:
        print(f"- {nome}")
elif 'df_final' not in locals() or df_final is None:
    print("DataFrame 'df_final' não encontrado. Execute as etapas anteriores.")
else:
    print("Coluna 'Nome de Curso' não encontrada no DataFrame 'df_final'.")

Valores únicos na coluna 'Nome de Curso' (df_final):
Total de valores únicos: 219
- Administração
- Agente Cultural
- Agente de Alimentação Escolar
- Agente de Assistência Técnica e Extensão Rural
- Agente de Desenvolvimento Cooperativista
- Agente de Gestão de Resíduos Sólidos
- Agente de Inclusão Digital em Centros Públicos de Acesso à Internet
- Agente de Projetos Sociais
- Agricultor Familiar
- Agroecologia
- Agroindústria
- Alemão Básico ­ A1
- Alimentos
- Análise e Desenvolvimento de Sistemas
- Aperfeiçoamento em Educação Ambiental e Justiça Climática No Nordeste
- Artesão em Bordado à Mão
- Assistente Administrativo
- Assistente de Costura
- Assistente de Dramaturgia
- Assistente de Secretaria Escolar
- Auxiliar de Cozinha
- Auxiliar de Manutenção Predial
- Camareira em Meios de Hospedagem
- Cinema e Audiovisual
- Ciências Biológicas
- Ciências Humanas e Saberes Contemporâneos para a Educação
- Ciências da Natureza
- Computação
- Comércio Exterior
- Condutor de Turismo em Unidad

In [11]:
# --- Função para aplicar os prefixos ---
def renomear_cursos(row):
    """
    Verifica o 'Tipo de Curso' em uma linha do DataFrame e adiciona o
    prefixo correspondente ao 'Nome de Curso'.
    """
    tipo_curso = row['Tipo de Curso']
    nome_curso = row['Nome de Curso']

    if tipo_curso == 'Licenciatura':
        return f'Licenciatura em {nome_curso}'
    elif tipo_curso == 'Qualificação Profissional (FIC)':
        # Adiciona o prefixo 'FIC em' em vez de 'Qualificação Profissional (FIC) em'
        return f'FIC {nome_curso}'
    elif tipo_curso == 'Tecnologia':
        return f'Tecnologia em {nome_curso}'
    else:
        # Se não for nenhum dos tipos especificados, retorna o nome original
        return nome_curso

# --- Criação da nova coluna 'Cursos' ---
# A função 'renomear_cursos' é aplicada a cada linha (axis=1) do DataFrame.
# O resultado é armazenado na nova coluna 'Cursos'.
df_final['Cursos'] = df_final.apply(renomear_cursos, axis=1)

# --- Exibição do DataFrame final com a nova coluna ---
print("--- DataFrame com a Nova Coluna 'Cursos' ---")
display(df_final)

--- DataFrame com a Nova Coluna 'Cursos' ---


,Co Inst,Instituição,Cod Unidade,Unidade de Ensino,Código da Unidade de Ensino - SISTEC,Código do Ciclo Matricula,Código da Matricula,Município,Código do Município com DV,UF,...,Vagas Extraordinárias AC,Vagas Extraordinárias l1,Vagas Extraordinárias l2,Vagas Extraordinárias l5,Vagas Extraordinárias l6,Vagas Extraordinárias l9,Vagas Extraordinárias l10,Vagas Extraordinárias l13,Vagas Extraordinárias l14,Cursos
0,26435,IFRN,346.0,Campus Nova Cruz,NaN,2449328,83355157,Nordeste,NaN,RN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tecnologia em Processos Químicos
1,26435,IFRN,346.0,Campus Nova Cruz,NaN,2449328,83354097,Nordeste,NaN,RN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tecnologia em Processos Químicos
2,26435,IFRN,346.0,Campus Nova Cruz,NaN,2449328,83354093,Nordeste,NaN,RN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tecnologia em Processos Químicos
3,26435,IFRN,338.0,Campus Natal Central,NaN,1781299,63886076,Nordeste,NaN,RN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tecnologia em Construção de Edifícios
4,26435,IFRN,346.0,Campus Nova Cruz,NaN,2449328,83354565,Nordeste,NaN,RN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tecnologia em Processos Químicos
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
369090,26435,IFRN,543.0,Campus Natal Zona Norte,2962.0,2676952,93465833,Natal,2408102.0,RN,...,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Técnico em Eletrônica
369091,26435,IFRN,543.0,Campus Natal Zona Norte,2962.0,2690046,93982603,Natal,2408102.0,RN,...,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Licenciatura em Informática
369092,26435,IFRN,168.0,Campus Ceará-Mirim,31313.0,2482770,84834793,Ceará-Mirim,2402600.0,RN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Técnico em Programação de Jogos Digitais
369093,26435,IFRN,312.0,Campus Macau,13016.0,2913155,107930046,Macau,2407203.0,RN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Licenciatura em Ciências Biológicas


In [12]:
# Exibir valores únicos da coluna 'Cursos' em df_final
if 'df_final' in locals() and df_final is not None and 'Cursos' in df_final.columns:
    unique_cursos = sorted(df_final['Cursos'].unique())
    print("Valores únicos na coluna 'Cursos' (df_final):")
    print(f"Total de valores únicos: {len(unique_cursos)}")
    for nome in unique_cursos:
        print(f"- {nome}")
elif 'df_final' not in locals() or df_final is None:
    print("DataFrame 'df_final' não encontrado. Execute as etapas anteriores.")
else:
    print("Coluna 'Cursos' não encontrada no DataFrame 'df_final'.")

Valores únicos na coluna 'Cursos' (df_final):
Total de valores únicos: 224
- Aperfeiçoamento em Educação Ambiental e Justiça Climática No Nordeste
- Ciências Humanas e Saberes Contemporâneos para a Educação
- Ciências da Natureza
- Curso de Pós-graduação Lato Sensu Especialização em Educação Ambiental e Geografia do Semi-árido
- Doutorado - Desenvolvimento Educacional e Social
- Doutorado Acadêmico em Educação Profissional
- Doutorado em Ensino da Rede Nordeste de Ensino
- Doutorado em Ensino da Rede Nordeste de Ensino (renoen)
- Engenharia Ambiental
- Engenharia Civil
- Engenharia Elétrica
- Engenharia de Produção
- Engenharia de Segurança do Trabalho
- Ensino de Ciências Naturais e Matemática
- Ensino de Geociências
- Ensino de Teatro
- Especialização (Lato Sensu) - Ambiente e Saúde
- Especialização (Lato Sensu) - Desenvolvimento Educacional e Social
- Especialização (Lato Sensu) - Gestão e Negócios
- Especialização (Lato Sensu) - Informação e Comunicação
- Especialização (Lato Sensu

In [13]:
print(f"Total de valores únicos na coluna 'Cursos' (df_final): {len(unique_cursos)}!!!")

Total de valores únicos na coluna 'Cursos' (df_final): 224!!!


In [14]:
# @title **ETAPA 6: Análise Exploratória e Limpeza dos Dados 'matriculas'**

def padronizar_nome_curso(df, nome_coluna='Cursos'):
    """
    Aplica uma série de regras de padronização a uma coluna de nomes de cursos em um DataFrame do pandas.

    As etapas de limpeza incluem:
    1. Remoção de informações redundantes (prefixos, sufixos de modalidade, etc.).
    2. Normalização de variações complexas e padronização de nomes específicos.
    3. Correção de erros de digitação e variações simples.
    4. Padronização da capitalização para Title Case, preservando conectivos em minúsculo.
    5. Remoção de espaços em branco extras.

    Args:
        df (pd.DataFrame): O DataFrame contendo os dados.
        nome_coluna (str): O nome da coluna com os nomes dos cursos a serem padronizados.

    Returns:
        pd.DataFrame: O DataFrame com a coluna de nomes de cursos padronizada.
    """
    # Copia a coluna para evitar alterar o DataFrame original durante o processo
    cursos_padronizados = df[nome_coluna].copy().astype(str)

    # --- Etapa 1: Remover informações redundantes (prefixos e sufixos) ---
    prefixos_para_remover = [
        r'^fic\s*-\s*',
        r'^pós graduação em\s*',
        r'^pós-graduação em\s*',
        r'^especialização \(lato sensu\)\s*-\s*',
        r'^especialização\s*-\s*',
        r'^mestrado\s*-\s*',
        r'^doutorado\s*-\s*',
        r'^mestrado profissional\s*-\s*',
        r'^fic qualificação profissional\s*-\s*',
        r'curso de Pós-graduação Lato Sensu '
    ]
    for prefixo in prefixos_para_remover:
        cursos_padronizados = cursos_padronizados.str.replace(prefixo, '', regex=True, flags=re.IGNORECASE)

    # Remove sufixos de modalidade
    sufixos_para_remover = [
        r',\s*na modalidade ead$',
        r',\s*na modalidade semipresencial$'
    ]
    for sufixo in sufixos_para_remover:
        cursos_padronizados = cursos_padronizados.str.replace(sufixo, '', regex=True, flags=re.IGNORECASE)

    # --- Etapa 2: Normalização de variações complexas ---
    # Este dicionário usa regex para encontrar padrões e substituir pelo nome padronizado.
    # A ordem é importante: regras mais específicas devem vir antes.
    regras_de_normalizacao = {
        # --- Doutorados ---
        # --- AJUSTE ---: Adicionada regra para Doutorado Acadêmico em Educação Profissional.
        r'.*Doutorado Acad[êe]mico em Educaç[ãa]o Profissional.*': 'Doutorado em Educação Profissional e Tecnológica',
        # --- AJUSTE ---: Adicionada regra para Doutorado em Desenvolvimento Educacional e Social.
        # r'.*Doutorado.*Desenvolvimento Educacional e Social.*': 'Doutorado em Desenvolvimento Educacional e Social',
        r'.*Doutorado em Ensino.*(Rede Nordeste|renoen).*': 'Doutorado em Ensino (RENOEN)',
        r'Rede Nordeste de Ensino - RENOEN': 'Doutorado em Ensino (RENOEN)',

        # --- Mestrados ---
        r'.*Mestrado.*Educação Profissional e Tecnológica.*': 'Mestrado em Educação Profissional e Tecnológica (ProfEPT)',
        r'Mestrado Acadêmico em Educação Profissional': 'Mestrado em Educação Profissional e Tecnológica (ProfEPT)',
        r'Mestrado em Ensino': 'Mestrado em Ensino (Posensino)',
        r'Mestrado Profissional - Recursos Naturais': 'Mestrado Profissional em Uso Sustentável dos Recursos Naturais',
        r'Uso Sustentável dos Recursos Naturais': 'Mestrado Profissional em Uso Sustentável dos Recursos Naturais',

        # --- Especializações ---
        r'Especialização em Educação Ambiental e Geografia do Semi-Árido': 'Especialização em Educação Ambiental e Geografia do Semiárido',
        r'Educação Ambiental e Geografia do Semiarido': 'Especialização em Educação Ambiental e Geografia do Semiárido',
        r'.*Ci[êe]ncias Humanas e Saberes Contempor[âa]neos.*': 'Especialização em Ciências Humanas e Saberes Contemporâneos para a Educação',
        r'Ciências Humanas e Competências Contemporâneas para a Educação': 'Especialização em Ciências Humanas e Competências Contemporâneas para a Educação',
        r'Docência para a Educação Profissional e Tecnológica (DocentEPT)': 'Especialização em Docência na Educação Profissional e Tecnológica (DocentEPT)',
        r'Especialização em Eja No Contexto da Diversidade': 'Especialização em Educação de Jovens e Adultos no Contexto da Diversidade',
        r'Engenharia de Segurança do Trabalho': 'Especialização em Engenharia de Segurança do Trabalho',
        r'Ensino de Ciências Biológicas': 'Especialização em Ensino de Ciências Biológicas',
        r'Ensino de Ciências Naturais e Matemática': 'Especialização em Ensino de Ciências Naturais e Matemática',
        r'Ensino de Geociências': 'Especialização em Ensino de Geociências',
        r'Ensino de Teatro': 'Especialização em Ensino de Teatro',
        r'Especialização em Ciência e Tecnologia de Alimentos, Na Modalidade Ead': 'Especialização em Ciência e Tecnologia de Alimentos',
        r'Especialização em Ensino de Língua Portuguesa e Matemática Transdisciplinar': 'Especialização em Ensino da Língua Portuguesa e Matemática numa Abordagem Transdisciplinar',
        r'Especialização em Docência para a Educação Profissional e Tecnológica – Docentept - Uab': 'Especialização em Docência na Educação Profissional e Tecnológica (DocentEPT)',
        r'Estudos Linguísticos e Literários': 'Especialização em Estudos Linguísticos e Literários',
        r'Especialização em Práticas Assertivas da Educação Profissional Integrada À Educação de Jovens e Adultos - Com Ênfase em Didática': 'Especialização em Práticas Assertivas em Didática e Gestão da Educação Profissional integrada à EJA',
        r'^Gestão Ambiental$': 'Especialização em Gestão Ambiental',
        r'Licenciatura em Licenciatura para a Educação Profissional, Científica e Tecnológica': 'Licenciatura em Formação Pedagógica para a Educação Básica, Profissional e Tecnológica'
     }

    for padrao, nome_correto in regras_de_normalizacao.items():
        cursos_padronizados = cursos_padronizados.str.replace(padrao, nome_correto, regex=True, flags=re.IGNORECASE)

    # --- Etapa 3: Corrigir erros de digitação e variações simples ---
    correcoes_simples = {
        'Conteporaneidade': 'Contemporaneidade',
        'À': 'à', # Corrige capitalização de crase
        'Semi-Árido': 'Semiárido',
        'Eja No Contexto da Diversidade': 'EJA no Contexto da Diversidade',
        'para O Ensino Médio': 'para o Ensino Médio',
        'desportiva e de Lazer': 'Desportiva e de Lazer'
    }
    for erro, correcao in correcoes_simples.items():
        cursos_padronizados = cursos_padronizados.str.replace(erro, correcao, regex=False)

    # --- Etapa 4: Padronização da capitalização ---
    # Converte para Title Case
    cursos_padronizados = cursos_padronizados.str.title()

    # Lista de conectivos e palavras curtas que devem ser minúsculas
    conectivos_minusculos = ['de', 'da', 'do', 'dos', 'das', 'em', 'e', 'para', 'o', 'a', 'à', 'os', 'as', 'no', 'na', 'nos', 'nas', 'com']

    def apply_title_case_with_exceptions(name):
        # Garante que siglas como IFRN, EJA, EAD, etc., fiquem em maiúsculas
        name = re.sub(r'\b(If(rn)?|Eja|Ead|Fic|Ppi|Pcd|Renoen|Profept|Docentept|Posensino)\b', lambda m: m.group().upper(), name, flags=re.IGNORECASE)
        words = name.split()
        processed_words = [word.lower() if word.lower() in conectivos_minusculos and i > 0 else word for i, word in enumerate(words)]
        return ' '.join(processed_words)

    cursos_padronizados = cursos_padronizados.apply(apply_title_case_with_exceptions)

    # --- Etapa 5: Remover espaços em branco extras ---
    cursos_padronizados = cursos_padronizados.str.strip()
    cursos_padronizados = cursos_padronizados.str.replace(r'\s+', ' ', regex=True)

    # Atribui a coluna padronizada de volta ao DataFrame
    df_processed = df.copy()
    df_processed[nome_coluna] = cursos_padronizados

    return df_processed

# Aplica a função de padronização à coluna 'Cursos'
df_processed = padronizar_nome_curso(df_final.copy(), 'Cursos')

if df_processed is not None:
    num_rows, num_cols = df_processed.shape
    print(f"DataFrame 'df_processed' criado como uma cópia de 'df_final' tem {num_rows:,} linhas e {num_cols} colunas.")
else:
    print("DataFrame 'df_processed' não encontrado. Execute as etapas anteriores.")

# Exibir valores únicos em ordem alfabética na coluna 'Cursos'
if df_processed is not None and 'Cursos' in df_processed.columns:
    unique_cursos = sorted(df_processed['Cursos'].unique())
    print("Valores únicos na coluna 'Cursos':")
    print(f"Total de valores únicos na coluna 'Cursos' após limpeza: {len(unique_cursos)}")
    for curso in unique_cursos:
        print(f"- {curso}")
elif df_processed is None:
    print("DataFrame 'df_processed' não encontrado. Execute as etapas anteriores.")
else:
    print("Coluna 'Cursos' não encontrada no DataFrame.")


DataFrame 'df_processed' criado como uma cópia de 'df_final' tem 369,095 linhas e 57 colunas.
Valores únicos na coluna 'Cursos':
Total de valores únicos na coluna 'Cursos' após limpeza: 194
- Ambiente e Saúde
- Aperfeiçoamento em Educação Ambiental e Justiça Climática no Nordeste
- Ciências da Natureza
- Controle e Processos Industriais
- Desenvolvimento Educacional e Social
- Doutorado em Educação Profissional e Tecnológica
- Doutorado em Ensino (RENOEN)
- Engenharia Ambiental
- Engenharia Civil
- Engenharia Elétrica
- Engenharia de Produção
- Especialização em Ciência e Tecnologia de Alimentos
- Especialização em Ciências Humanas e Saberes Contemporâneos para a Educação
- Especialização em Docência na Educação Profissional e Tecnológica (DOCENTEPT)
- Especialização em Educação Ambiental e Geografia do Semiárido
- Especialização em Educação Profissional
- Especialização em Educação de Jovens e Adultos no Contexto da Diversidade
- Especialização em Educação e Contemporaneidade
- Especi

In [15]:
print(f"Total de valores únicos na coluna 'Cursos' após limpeza: {len(unique_cursos)}")

Total de valores únicos na coluna 'Cursos' após limpeza: 194


In [16]:
# @title **Mapear os valores em 'Matrícula Atendida' e converter para `booleano`**
if 'df_processed' in locals() and df_processed is not None and 'Matrícula Atendida' in df_processed.columns:
    print("Convertendo a coluna 'Matrícula Atendida' para booleano...")
    mapping = {"Sim": 1, "Y": 1, "Não Informado": 0}
    # Ensure the column is object type to handle mixed types before mapping
    df_processed['Matrícula Atendida'] = df_processed['Matrícula Atendida'].astype(str).map(mapping).astype(bool)
    print("Conversão concluída.")
    print("\nContagem dos valores na coluna 'Matrícula Atendida' após conversão:")
    display(df_processed['Matrícula Atendida'].value_counts())
elif 'df_processed' not in locals() or df_processed is None:
    print("DataFrame 'df_processed' não encontrado. Execute as etapas anteriores.")
else:
    print("Coluna 'Matrícula Atendida' não encontrada no DataFrame 'df_processed'.")

Convertendo a coluna 'Matrícula Atendida' para booleano...
Conversão concluída.

Contagem dos valores na coluna 'Matrícula Atendida' após conversão:


,count
Matrícula Atendida,
True,280752
False,88343


In [17]:
# @title Ajustar parâmetros específicos em diversas colunas'
# Corrige parâmetros na coluna 'Eixo Tecnológico'
replacements_eixo = {
    'GESTÃO E NEGÓCIOS': 'Gestão e Negócios',
    'DESENVOLVIMENTO EDUCACIONAL E SOCIAL': 'Desenvolvimento Educacional e Social'
}
if 'Eixo Tecnológico' in df_processed.columns:
  df_processed['Eixo Tecnológico'] = df_processed['Eixo Tecnológico'].replace(replacements_eixo)

# Corrige parâmetros na coluna 'Subeixo Tecnológico'
replacements_subeixo = {
    'AMBIENTE E SAÚDE': 'Ambiente e Saúde'
}
if 'Subeixo Tecnológico' in df_processed.columns:
  df_processed['Subeixo Tecnológico'] = df_processed['Subeixo Tecnológico'].replace(replacements_eixo)

# Remove 'Campus ' and 'Campus Avançado ' de 'Unidade de Ensino'
if 'Unidade de Ensino' in df_processed.columns:
    df_processed['Unidade de Ensino'] = df_processed['Unidade de Ensino'].str.replace('Campus Avançado ', '', regex=False)
    df_processed['Unidade de Ensino'] = df_processed['Unidade de Ensino'].str.replace('Campus ', '', regex=False)

# Substitui valores na coluna 'Forma de ingresso'
replacements_ingresso = {
    'AC': 'AC: Ampla Concorrência',
    'l1': 'L1: Renda Baixa',
    'l2': 'L2: Renda Baixa + PPI',
    'l5': 'L5: Ensino Médio em Escolas Pública',
    'l6': 'L6: PPI (Pretos, Pardos e Indígenas)',
    'l9': 'L9: Renda Baixa + PcD',
    'l10': 'L10: Renda Baixa + PPI + PcD',
    'l13': 'L13: PcD (Independente de Renda)',
    'l14': 'L14: PcD + PPI (Independente de Renda)',
    'LB_EP': 'LB_EP: Baixa Renda + Ensino Médio em Escola Pública',
    'LB_PCD': 'LB_PCD: Baixa Renda + PcD',
    'LB_PPI': 'LB_PPI: Baixa Renda + PPI',
    'LB_Q': 'LB_Q: Baixa Renda + Quilombolas + Ensino Médio em Escola Pública',
    'LI_EP': 'LI_EP: Ensino Médio em Escola Pública (Independentemente da Renda)',
    'LI_PCD': 'LI_PCD: PcD (Independente de Renda)',
    'LI_PPI': 'LI_PPI: PPI (Independente de Renda)',
    'LI_Q': 'LI_Q: Quilombolas + Ensino Médio em Escola Pública (Independente da Renda)',
    'nan': 'NI: Não informado'
}

if 'Forma de ingresso' in df_processed.columns:
    # Certifica que a coluna seja do tipo objeto para lidar com substituições de strings, incluindo 'nan'
    df_processed['Forma de ingresso'] = df_processed['Forma de ingresso'].astype(str).replace(replacements_ingresso)

# Renomeia colunas adequadamente para o BigQuery
df_processed = df_processed.rename(columns={
    'Carga Horaria': 'Carga_Horaria',
    'Carga Horaria Mínima': 'Carga_Horaria_Minima',
    'Categoria da Situação': 'Categoria_da_Situacao',
    'Co Inst': 'Co_Inst',
    'Cod Unidade': 'Cod_Unidade',
    'Cor / Raça': 'Cor_Raça',
    'Código da Matricula': 'Codigo_da_Matricula',
    'Código da Unidade de Ensino - SISTEC': 'Codigo_da_Unidade_de_Ensino_-_SISTEC',
    'Código do Ciclo Matricula': 'Codigo_do_Ciclo_Matricula',
    'Código do Município com DV': 'Codigo_do_Municipio_com_DV',
    'Data de Fim Previsto do Ciclo': 'Data_de_Fim_Previsto_do_Ciclo',
    'Data de Inicio do Ciclo': 'Data_de_Inicio_do_Ciclo',
    'Data de Ocorrencia da Matricula': 'Data_de_Ocorrencia_da_Matricula',
    'Situação de Matrícula': 'Situacao_de_Matricula',
    'Subeixo Tecnológico': 'Subeixo_Tecnologico',
    'Matrícula Atendida': 'Matricula_Atendida',
    'Eixo Tecnológico': 'Eixo_Tecnologico',
    'Faixa Etária': 'Faixa_Etaria',
    'Fator Esforço Curso': 'Fator_Esforco_Curso',
    'Fonte de Financiamento': 'Fonte_de_Financiamento',
    'Forma de ingresso': 'Forma_de_Ingresso',
    'Instituição': 'Instituicao',
    'Mês De Ocorrência da Situação': 'Mes_De_Ocorrencia_da_Situacao',
    'Município': 'Municipio',
    'Modalidade de Ensino': 'Modalidade_Educacional',
    'Tipo de Curso': 'Nivel_Educacional',
    'Tipo de Oferta': 'Modalidade_de_Curso',
    'Nome de Curso': 'Nome_de_Curso',
    'Total de Inscritos': 'Total_de_Inscritos',
    'Região': 'Regiao',
    'Renda Familiar': 'Renda_Familiar',
    'Sexo': 'Sexo',
    'Unidade de Ensino': 'Campus_do_IFRN',
    'Vagas Extraordinárias AC': 'Vagas_Extraordinarias_AC',
    'Vagas Extraordinárias l1': 'Vagas_Extraordinarias_l1',
    'Vagas Extraordinárias l10': 'Vagas_Extraordinarias_l10',
    'Vagas Extraordinárias l13': 'Vagas_Extraordinarias_l13',
    'Vagas Extraordinárias l14': 'Vagas_Extraordinarias_l14',
    'Vagas Extraordinárias l2': 'Vagas_Extraordinarias_l2',
    'Vagas Extraordinárias l5': 'Vagas_Extraordinarias_l5',
    'Vagas Extraordinárias l6': 'Vagas_Extraordinarias_l6',
    'Vagas Extraordinárias l9': 'Vagas_Extraordinarias_l9',
    'Vagas Ofertadas': 'Vagas_Ofertadas',
    'Vagas Regulares AC': 'Vagas_Regulares_AC',
    'Vagas Regulares l1': 'Vagas_Regulares_l1',
    'Vagas Regulares l10': 'Vagas_Regulares_l10',
    'Vagas Regulares l13': 'Vagas_Regulares_l13',
    'Vagas Regulares l14': 'Vagas_Regulares_l14',
    'Vagas Regulares l2': 'Vagas_Regulares_l2',
    'Vagas Regulares l5': 'Vagas_Regulares_l5',
    'Vagas Regulares l6': 'Vagas_Regulares_l6',
    'Vagas Regulares l9': 'Vagas_Regulares_l9',
})

# Exibir informações sobre as colunas e tipos de dados de 'df_processed'
if df_processed is not None:
    print("Informações sobre as colunas e tipos de dados do DataFrame 'df_processed':")
    df_processed.info()
else:
    print("DataFrame 'df_processed' não encontrado. Execute as etapas anteriores.")

Informações sobre as colunas e tipos de dados do DataFrame 'df_processed':
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 369095 entries, 0 to 369094
Data columns (total 57 columns):
 #   Column                                Non-Null Count   Dtype  
---  ------                                --------------   -----  
 0   Co_Inst                               369095 non-null  int64  
 1   Instituicao                           369095 non-null  object 
 2   Cod_Unidade                           369095 non-null  float64
 3   Campus_do_IFRN                        369095 non-null  object 
 4   Codigo_da_Unidade_de_Ensino_-_SISTEC  307975 non-null  float64
 5   Codigo_do_Ciclo_Matricula             369095 non-null  int64  
 6   Codigo_da_Matricula                   369095 non-null  int64  
 7   Municipio                             369095 non-null  object 
 8   Codigo_do_Municipio_com_DV            308474 non-null  float64
 9   UF                                    369095 non-null  object

In [18]:
# @title **Tratamento dos tipos de dados das colunas**
def converter_tipos_de_dados(df: pd.DataFrame, colunas_para_int: list, colunas_para_str: list, colunas_para_data: dict, colunas_para_float: dict) -> pd.DataFrame:
    """
    Converte os tipos de dados de colunas especificadas em um DataFrame.

    Args:
        df (pd.DataFrame): O DataFrame a ser modificado.
        colunas_para_int (list): Uma lista de nomes de colunas para converter para o tipo Int64 (inteiro que suporta nulos).
        colunas_para_str (list): Uma lista de nomes de colunas para converter para o tipo string.
        colunas_para_data (dict): Um dicionário onde as chaves são os nomes das colunas
                                  para converter para data e os valores são os formatos esperados (ex: {'coluna': '%d/%m/%Y'}).
        colunas_para_float (dict): Um dicionário onde as chaves são os nomes das colunas
                                   para converter para float e os valores são os caracteres a serem substituídos
                                   antes da conversão (ex: {'coluna': ','}).

    Returns:
        pd.DataFrame: Uma cópia do DataFrame com os tipos de dados convertidos.
    """
    if not isinstance(df, pd.DataFrame):
        print("Erro: O objeto fornecido não é um DataFrame do pandas.")
        return None

    # Cria uma cópia para evitar modificar o DataFrame original
    df_processado = df.copy()

    print("--- Iniciando conversão de tipos de dados ---")

    # --- Conversão para Inteiro (Int64) ---
    print("\nConvertendo colunas para Inteiro:")
    for col in colunas_para_int:
        if col in df_processado.columns:
            try:
                # Converte para float primeiro para lidar com NaN, depois para Int64 (que suporta NaN)
                df_processado[col] = pd.to_numeric(df_processado[col], errors='coerce').astype('Int64')
                print(f"  ✔ Coluna '{col}' convertida para Int64.")
            except Exception as e:
                print(f"  ❌ Erro ao converter a coluna '{col}': {e}")
        else:
            print(f"  - AVISO: Coluna '{col}' não encontrada no DataFrame.")

    # --- Conversão para String ---
    print("\nConvertendo colunas para String:")
    for col in colunas_para_str:
        if col in df_processado.columns:
            try:
                # Converte valores nulos para uma string vazia antes de mudar o tipo
                df_processado[col] = df_processado[col].fillna('').astype(str)
                print(f"  ✔ Coluna '{col}' convertida para string.")
            except Exception as e:
                print(f"  ❌ Erro ao converter a coluna '{col}': {e}")
        else:
            print(f"  - AVISO: Coluna '{col}' não encontrada no DataFrame.")

    # --- Conversão para Data ---
    print("\nConvertendo colunas para Data:")
    for col, fmt in colunas_para_data.items():
        if col in df_processado.columns:
            try:
                df_processado[col] = pd.to_datetime(df_processado[col], format=fmt, errors='coerce')
                print(f"  ✔ Coluna '{col}' convertida para data com formato '{fmt}'.")
            except Exception as e:
                print(f"  ❌ Erro ao converter a coluna '{col}' para data: {e}")
        else:
            print(f"  - AVISO: Coluna '{col}' não encontrada no DataFrame.")

    # --- Conversão para Float (float64) ---
    print("\nConvertendo colunas para Float:")
    for col, char_to_replace in colunas_para_float.items():
        if col in df_processado.columns:
            try:
                # Substitui o caractere e converte para numérico
                df_processado[col] = df_processado[col].astype(str).str.replace(char_to_replace, '.', regex=False)
                df_processado[col] = pd.to_numeric(df_processado[col], errors='coerce').astype('float64')
                print(f"  ✔ Coluna '{col}' convertida para float64.")
            except Exception as e:
                print(f"  ❌ Erro ao converter a coluna '{col}' para float64: {e}")
        else:
            print(f"  - AVISO: Coluna '{col}' não encontrada no DataFrame.")

    print("\n--- Conversão concluída ---")
    return df_processado

# --- Exemplo de como usar a função ---

# Supondo que 'df_processed' seja o seu DataFrame
# df_processed = pd.read_csv(...) # Carregue seus dados aqui

# Define as listas de colunas
colunas_int = [
    'Carga_Horaria_Minima', 'Idade', 'Total_de_Inscritos',
    'Vagas_Extraordinarias_AC', 'Vagas_Extraordinarias_l1', 'Vagas_Extraordinarias_l10',
    'Vagas_Extraordinarias_l13', 'Vagas_Extraordinarias_l14', 'Vagas_Extraordinarias_l2',
    'Vagas_Extraordinarias_l5', 'Vagas_Extraordinarias_l6', 'Vagas_Extraordinarias_l9',
    'Vagas_Ofertadas', 'Vagas_Regulares_AC', 'Vagas_Regulares_l1', 'Vagas_Regulares_l10',
    'Vagas_Regulares_l13', 'Vagas_Regulares_l14', 'Vagas_Regulares_l2', 'Vagas_Regulares_l5',
    'Vagas_Regulares_l6', 'Vagas_Regulares_l9'
]
colunas_str = [
    'Cod_Unidade',
    'Co_Inst',
    'Codigo_da_Matricula',
    'Codigo_da_Unidade_de_Ensino_-_SISTEC',
    'Codigo_do_Ciclo_Matricula',
    'Codigo_do_Municipio_com_DV',
    'Eixo_Tecnologico',
    'Forma_de_Ingresso',
    'Instituicao',
    'Modalidade_Educacional',
    'Nivel_Educacional',
    'Modalidade_de_Curso',
    'Matricula_Atendida',
    'Renda_Familiar',
    'Subeixo_Tecnologico',
    'Categoria_da_Situacao',
]
colunas_data = {
    'Data_de_Fim_Previsto_do_Ciclo': '%d/%m/%Y',
    'Data_de_Inicio_do_Ciclo': '%d/%m/%Y',
    'Data_de_Ocorrencia_da_Matricula': '%d/%m/%Y'
}

colunas_float = {
    'Fator_Esforco_Curso': ',' # Substituir vírgula por ponto
}

# Chama a função e armazena o resultado em um novo DataFrame
df_processed = converter_tipos_de_dados(df_processed, colunas_int, colunas_str, colunas_data, colunas_float)

# Verifica os tipos de dados após a conversão (opcional)
if df_processed is not None:
  print("\nTipos de dados após a conversão:")
  colunas_convertidas = colunas_int + colunas_str + list(colunas_data.keys()) + list(colunas_float.keys())
  print(df_processed[colunas_convertidas].dtypes)

--- Iniciando conversão de tipos de dados ---

Convertendo colunas para Inteiro:
  ✔ Coluna 'Carga_Horaria_Minima' convertida para Int64.
  ✔ Coluna 'Idade' convertida para Int64.
  ✔ Coluna 'Total_de_Inscritos' convertida para Int64.
  ✔ Coluna 'Vagas_Extraordinarias_AC' convertida para Int64.
  ✔ Coluna 'Vagas_Extraordinarias_l1' convertida para Int64.
  ✔ Coluna 'Vagas_Extraordinarias_l10' convertida para Int64.
  ✔ Coluna 'Vagas_Extraordinarias_l13' convertida para Int64.
  ✔ Coluna 'Vagas_Extraordinarias_l14' convertida para Int64.
  ✔ Coluna 'Vagas_Extraordinarias_l2' convertida para Int64.
  ✔ Coluna 'Vagas_Extraordinarias_l5' convertida para Int64.
  ✔ Coluna 'Vagas_Extraordinarias_l6' convertida para Int64.
  ✔ Coluna 'Vagas_Extraordinarias_l9' convertida para Int64.
  ✔ Coluna 'Vagas_Ofertadas' convertida para Int64.
  ✔ Coluna 'Vagas_Regulares_AC' convertida para Int64.
  ✔ Coluna 'Vagas_Regulares_l1' convertida para Int64.
  ✔ Coluna 'Vagas_Regulares_l10' convertida para In

In [19]:
# @title **Exibir estatísticas básicas para colunas selecionadas de `df_processed`**
if df_processed is not None and not df_processed.empty:
    columns_to_describe = [
        'Idade',
        'Total_de_Inscritos',
        'Vagas_Extraordinarias_AC',
        'Vagas_Extraordinarias_l1',
        'Vagas_Extraordinarias_l10',
        'Vagas_Extraordinarias_l13',
        'Vagas_Extraordinarias_l14',
        'Vagas_Extraordinarias_l2',
        'Vagas_Extraordinarias_l5',
        'Vagas_Extraordinarias_l6',
        'Vagas_Extraordinarias_l9',
        'Vagas_Ofertadas',
        'Vagas_Regulares_AC',
        'Vagas_Regulares_l1',
        'Vagas_Regulares_l10',
        'Vagas_Regulares_l13',
        'Vagas_Regulares_l14',
        'Vagas_Regulares_l2',
        'Vagas_Regulares_l5',
        'Vagas_Regulares_l6',
        'Vagas_Regulares_l9',
    ]

    # Filter only columns that exist in the DataFrame
    existing_columns = [col for col in columns_to_describe if col in df_processed.columns]

    if existing_columns:
        print("Estatísticas Descritivas para Colunas Selecionadas:")
        # Try to describe numerical columns
        try:
            # Set float format option for display
            pd.options.display.float_format = '{:,.4f}'.format
            display(df_processed[existing_columns].describe())
        except Exception as e:
            print(f"Error generating statistics for selected numerical columns: {e}")

        # Try to describe object columns (if any)
        try:
            object_columns = df_processed[existing_columns].select_dtypes(include='object').columns.tolist()
            if object_columns:
                 print("\nEstatísticas Descritivas para Colunas de Objeto Selecionadas:")
                 display(df_processed[object_columns].describe())
        except Exception as e:
             print(f"Error generating statistics for selected object columns: {e}")

        # Remove float format option after display, if desired so as not to affect other displays
        # pd.options.display.float_format = None

    else:
        print("None of the selected columns were found in the DataFrame.")

elif df_processed is not None and df_processed.empty:
    print("DataFrame 'df_processed' is empty. No statistics to display.")
else:
    print("DataFrame 'df_processed' not found. Execute previous steps.")

Estatísticas Descritivas para Colunas Selecionadas:


,Idade,Total_de_Inscritos,Vagas_Extraordinarias_AC,Vagas_Extraordinarias_l1,Vagas_Extraordinarias_l10,Vagas_Extraordinarias_l13,Vagas_Extraordinarias_l14,Vagas_Extraordinarias_l2,Vagas_Extraordinarias_l5,Vagas_Extraordinarias_l6,...,Vagas_Ofertadas,Vagas_Regulares_AC,Vagas_Regulares_l1,Vagas_Regulares_l10,Vagas_Regulares_l13,Vagas_Regulares_l14,Vagas_Regulares_l2,Vagas_Regulares_l5,Vagas_Regulares_l6,Vagas_Regulares_l9
count,"369,079.0000","316,419.0000","227,657.0000","218,393.0000","218,224.0000","218,213.0000","218,223.0000","218,501.0000","218,289.0000","218,377.0000",...,"88,902.0000","259,892.0000","239,911.0000","238,015.0000","238,418.0000","237,935.0000","239,895.0000","239,956.0000","240,049.0000","194,148.0000"
mean,24.4939,424.3934,2.5344,0.0218,0.0013,0.0034,0.0009,0.0193,0.0101,0.0207,...,163.7168,102.7219,2.6431,0.9748,0.8635,0.9770,4.6452,15.9344,4.7564,0.8484
std,9.9615,652.0633,15.9306,0.6645,0.0364,0.0826,0.0307,0.2147,0.2454,0.2178,...,408.5238,289.4063,5.4394,1.1984,1.0577,1.2025,4.1405,82.4175,5.1383,1.0489
min,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
25%,18.0000,97.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,40.0000,20.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
50%,21.0000,236.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,42.0000,20.0000,3.0000,1.0000,1.0000,1.0000,5.0000,3.0000,5.0000,1.0000
75%,29.0000,470.0000,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,81.0000,40.0000,4.0000,2.0000,1.0000,2.0000,6.0000,4.0000,6.0000,1.0000
max,118.0000,"7,377.0000",306.0000,34.0000,1.0000,2.0000,1.0000,4.0000,9.0000,9.0000,...,"2,439.0000","2,439.0000",179.0000,12.0000,12.0000,12.0000,28.0000,913.0000,134.0000,12.0000


In [20]:
# @title **Análise Estatística de Outliers**

def analyze_outliers_iqr(df: pd.DataFrame, columns: list):
    """
    Calcula e exibe os limites superior e inferior para identificação de outliers
    usando o método IQR para as colunas numéricas especificadas.

    Args:
        df (pd.DataFrame): O DataFrame a ser analisado.
        columns (list): Uma lista de nomes de colunas numéricas para análise.
    """
    if df is None or df.empty:
        print("DataFrame vazio ou não encontrado. Nenhuma análise de outlier realizada.")
        return

    print("--- Análise de Outliers (Método IQR) ---")

    for col in columns:
        if col in df.columns and pd.api.types.is_numeric_dtype(df[col]):
            # Remove valores nulos para o cálculo do IQR
            data = df[col].dropna()

            if len(data) < 4: # Precisa de pelo menos 4 pontos para calcular Q1, Q3 e IQR
                print(f"  - Coluna '{col}': Não há dados suficientes para análise de outlier.")
                continue

            Q1 = data.quantile(0.25)
            Q3 = data.quantile(0.75)
            IQR = Q3 - Q1

            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR

            print(f"\n--- Coluna: '{col}' ---")
            print(f"  Q1 (25º Percentil): {Q1:,.4f}")
            print(f"  Q3 (75º Percentil): {Q3:,.4f}")
            print(f"  IQR (Intervalo Interquartil): {IQR:,.4f}")
            print(f"  Limite Inferior (Q1 - 1.5*IQR): {lower_bound:,.4f}")
            print(f"  Limite Superior (Q3 + 1.5*IQR): {upper_bound:,.4f}")

            # Opcional: Contar e exibir outliers
            outliers_lower = data[data < lower_bound]
            outliers_upper = data[data > upper_bound]
            total_outliers = len(outliers_lower) + len(outliers_upper)

            print(f"  Número de potenciais outliers abaixo do limite inferior: {len(outliers_lower):,}")
            print(f"  Número de potenciais outliers acima do limite superior: {len(outliers_upper):,}")
            print(f"  Total de potenciais outliers: {total_outliers:,}")

        elif col in df.columns:
             print(f"  - Coluna '{col}': Não é um tipo de dado numérico. Pulando análise de outlier.")
        else:
            print(f"  - AVISO: Coluna '{col}' não encontrada no DataFrame.")

# Lista de colunas para analisar outliers
columns_for_outlier_analysis = [
    'Idade',
    'Total_de_Inscritos',
    'Vagas_Extraordinarias_AC',
    'Vagas_Extraordinarias_l1',
    'Vagas_Extraordinarias_l10',
    'Vagas_Extraordinarias_l13',
    'Vagas_Extraordinarias_l14',
    'Vagas_Extraordinarias_l2',
    'Vagas_Extraordinarias_l5',
    'Vagas_Extraordinarias_l6',
    'Vagas_Extraordinarias_l9',
    'Vagas_Ofertadas',
    'Vagas_Regulares_AC',
    'Vagas_Regulares_l1',
    'Vagas_Regulares_l10',
    'Vagas_Regulares_l13',
    'Vagas_Regulares_l14',
    'Vagas_Regulares_l2',
    'Vagas_Regulares_l5',
    'Vagas_Regulares_l6',
    'Vagas_Regulares_l9',
]

# Executa a análise
analyze_outliers_iqr(df_processed, columns_for_outlier_analysis)

--- Análise de Outliers (Método IQR) ---

--- Coluna: 'Idade' ---
  Q1 (25º Percentil): 18.0000
  Q3 (75º Percentil): 29.0000
  IQR (Intervalo Interquartil): 11.0000
  Limite Inferior (Q1 - 1.5*IQR): 1.5000
  Limite Superior (Q3 + 1.5*IQR): 45.5000
  Número de potenciais outliers abaixo do limite inferior: 6
  Número de potenciais outliers acima do limite superior: 17,789
  Total de potenciais outliers: 17,795

--- Coluna: 'Total_de_Inscritos' ---
  Q1 (25º Percentil): 97.0000
  Q3 (75º Percentil): 470.0000
  IQR (Intervalo Interquartil): 373.0000
  Limite Inferior (Q1 - 1.5*IQR): -462.5000
  Limite Superior (Q3 + 1.5*IQR): 1,029.5000
  Número de potenciais outliers abaixo do limite inferior: 0
  Número de potenciais outliers acima do limite superior: 23,119
  Total de potenciais outliers: 23,119

--- Coluna: 'Vagas_Extraordinarias_AC' ---
  Q1 (25º Percentil): 0.0000
  Q3 (75º Percentil): 1.0000
  IQR (Intervalo Interquartil): 1.0000
  Limite Inferior (Q1 - 1.5*IQR): -1.5000
  Limite 

In [21]:
# Exibir informações sobre as colunas e tipos de dados de 'df_processed'
if df_processed is not None:
    print("Informações sobre as colunas e tipos de dados do DataFrame 'df_processed':")
    df_processed.info()
else:
    print("DataFrame 'df_processed' não encontrado. Execute as etapas anteriores.")

Informações sobre as colunas e tipos de dados do DataFrame 'df_processed':
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 369095 entries, 0 to 369094
Data columns (total 57 columns):
 #   Column                                Non-Null Count   Dtype         
---  ------                                --------------   -----         
 0   Co_Inst                               369095 non-null  object        
 1   Instituicao                           369095 non-null  object        
 2   Cod_Unidade                           369095 non-null  object        
 3   Campus_do_IFRN                        369095 non-null  object        
 4   Codigo_da_Unidade_de_Ensino_-_SISTEC  369095 non-null  object        
 5   Codigo_do_Ciclo_Matricula             369095 non-null  object        
 6   Codigo_da_Matricula                   369095 non-null  object        
 7   Municipio                             369095 non-null  object        
 8   Codigo_do_Municipio_com_DV            369095 non-null  obje

In [22]:
# @title **ETAPA 7: Exportação para o BigQuery**

# --- Configurações de Destino do BigQuery ---
import pandas_gbq

project_id = "pnp-data-extraction" # Substitua pelo ID do seu projeto
dataset_id = "pnp_dados_IFRN"      # Nome do conjunto de dados

# O nome da tabela no BigQuery será o mesmo nome da tabela da PNP
table_id = f"df_{table_name}"
destination_table = f"{dataset_id}.{table_id}"

# --- Execução da Exportação ---
if df_processed is not None and not df_processed.empty:
    print(f"Iniciando a exportação de {len(df_processed):,} linhas para o BigQuery...")
    print(f"Destino: {project_id}.{destination_table}")

    # --- Renomear colunas para serem compatíveis com BigQuery ---
    # Substitui espaços e '/' por '_'
    df_processed_bq = df_processed.copy()
    df_processed_bq.columns = (df_processed_bq.columns
            .str.replace(' ', '_', regex=False)
            .str.replace('/', '_', regex=False)
            .str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
        )
    print("Nomes das colunas padronizados para o BigQuery.")

    # --- Definição do Esquema da Tabela ---
    # Defina o nome e o tipo de cada coluna que você quer controlar.
    # Tipos comuns: 'STRING', 'INTEGER', 'FLOAT', 'NUMERIC', 'BOOLEAN', 'TIMESTAMP', 'DATE'.
    table_schema = [
        {"name": "Co_Inst", "type": "STRING"},
        {"name": "Instituicao", "type": "STRING"},
        {"name": "Cod_Unidade", "type": "STRING"},
        {"name": "Campus_do_IFRN", "type": "STRING"},
        {"name": "Codigo_da_Unidade_de_Ensino_-_SISTEC", "type": "STRING"},
        {"name": "Codigo_do_Ciclo_Matricula", "type": "STRING"},
        {"name": "Codigo_da_Matricula", "type": "STRING"},
        {"name": "Municipio", "type": "STRING"},
        {"name": "Codigo_do_Municipio_com_DV", "type": "STRING"},
        {"name": "UF", "type": "STRING"},
        {"name": "Regiao", "type": "STRING"},
        {"name": "Modalidade_Educacional", "type": "STRING"},
        {"name": "Nivel_Educacional", "type": "STRING"},
        {"name": "Modalidade_de_Curso", "type": "STRING"},
        {"name": "Eixo_Tecnologico", "type": "STRING"},
        {"name": "Subeixo_Tecnologico", "type": "STRING"},
        {"name": "Nome_de_Curso", "type": "STRING"},
        {"name": "Cursos", "type": "STRING"},
        {"name": "Turno", "type": "STRING"},
        {"name": "Carga_Horaria", "type": "INTEGER"},
        {"name": "Carga_Horaria_Minima", "type": "INTEGER"},
        {"name": "Fonte_de_Financiamento", "type": "STRING"},
        {"name": "Fator_Esforco_Curso", "type": "FLOAT"},
        {"name": "Sexo", "type": "STRING"},
        {"name": "Cor_Raça", "type": "STRING"},
        {"name": "Idade", "type": "INTEGER"},
        {"name": "Faixa_Etaria", "type": "STRING"},
        {"name": "Renda_Familiar", "type": "STRING"},
        {"name": "Ano", "type": "INTEGER"},
        {"name": "Matricula_Atendida", "type": "STRING"},
        {"name": "Situacao_de_Matricula", "type": "STRING"},
        {"name": "Categoria_da_Situacao", "type": "STRING"},
        {"name": "Data_de_Inicio_do_Ciclo", "type": "DATETIME"},
        {"name": "Data_de_Fim_Previsto_do_Ciclo", "type": "DATETIME"},
        {"name": "Data_de_Ocorrencia_da_Matricula", "type": "DATETIME"},
        {"name": "Mes_De_Ocorrencia_da_Situacao", "type": "STRING"},
        {"name": "Total_de_Inscritos", "type": "INTEGER"},
        {"name": "Forma_de_Ingresso", "type": "STRING"},
        {"name": "Vagas_Ofertadas", "type": "INTEGER"},
        {"name": "Vagas_Regulares_AC", "type": "INTEGER"},
        {"name": "Vagas_Regulares_l1", "type": "INTEGER"},
        {"name": "Vagas_Regulares_l2", "type": "INTEGER"},
        {"name": "Vagas_Regulares_l5", "type": "INTEGER"},
        {"name": "Vagas_Regulares_l6", "type": "INTEGER"},
        {"name": "Vagas_Regulares_l9", "type": "INTEGER"},
        {"name": "Vagas_Regulares_l10", "type": "INTEGER"},
        {"name": "Vagas_Regulares_l13", "type": "INTEGER"},
        {"name": "Vagas_Regulares_l14", "type": "INTEGER"},
        {"name": "Vagas_Extraordinarias_AC", "type": "INTEGER"},
        {"name": "Vagas_Extraordinarias_l1", "type": "INTEGER"},
        {"name": "Vagas_Extraordinarias_l2", "type": "INTEGER"},
        {"name": "Vagas_Extraordinarias_l5", "type": "INTEGER"},
        {"name": "Vagas_Extraordinarias_l6", "type": "INTEGER"},
        {"name": "Vagas_Extraordinarias_l9", "type": "INTEGER"},
        {"name": "Vagas_Extraordinarias_l10", "type": "INTEGER"},
        {"name": "Vagas_Extraordinarias_l13", "type": "INTEGER"},
        {"name": "Vagas_Extraordinarias_l14", "type": "INTEGER"}
    ]
    # -----------------------------------------------------------------

    # Envia o DataFrame para o BigQuery com o esquema especificado
    try:
        # Usando a função recomendada pandas_gbq.to_gbq
        pandas_gbq.to_gbq(
            df_processed_bq, # Exporta o DataFrame com colunas renomeadas
            destination_table=destination_table,
            project_id=project_id,
            if_exists='replace',  # Opções: 'fail', 'replace', 'append'
            progress_bar=True
        )
        print(f"\n✔ DataFrame exportado com sucesso para o BigQuery!")
        print(f"Link para a tabela: https://console.cloud.google.com/bigquery?project={project_id}&ws=!1m5!1m4!4m3!1s{project_id}!2s{dataset_id}!3s{table_id}")
    except Exception as e:
        print(f"❌ ERRO durante a exportação para o BigQuery: {e}")

elif df_processed is not None and df_processed.empty:
    print("AVISO: O DataFrame final está vazio. Nenhuma exportação foi realizada.")
else:
    print("❌ ERRO: O DataFrame a ser exportado não foi encontrado. A exportação foi cancelada.")

Iniciando a exportação de 369,095 linhas para o BigQuery...
Destino: pnp-data-extraction.pnp_dados_IFRN.df_matriculas
Nomes das colunas padronizados para o BigQuery.


100%|██████████| 1/1 [00:00<00:00, 13189.64it/s]


✔ DataFrame exportado com sucesso para o BigQuery!
Link para a tabela: https://console.cloud.google.com/bigquery?project=pnp-data-extraction&ws=!1m5!1m4!4m3!1spnp-data-extraction!2spnp_dados_IFRN!3sdf_matriculas
